In [31]:
import cv2
from cv2 import aruco
import numpy as np
import glob

# Charuco board variables
CHARUCOBOARD_ROWCOUNT = 17
CHARUCOBOARD_COLCOUNT = 24
ARUCO_DICT = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)
CHARUCO_BOARD = aruco.CharucoBoard(
    size=(CHARUCOBOARD_COLCOUNT, CHARUCOBOARD_ROWCOUNT),
    squareLength=0.0075,
    markerLength=0.005,
    dictionary=ARUCO_DICT)

# Glob images
images = glob.glob('./Images/*.jpg')

# Arrays to store Charuco corners and IDs for all images
allCorners = []
allIds = []

# Loop over images
for fname in images:
    # Load image
    img = cv2.imread(fname)
    # Resize image
    img = cv2.resize(img, (0, 0), fx=0.3, fy=0.3)
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find aruco markers in the query image
    corners, ids, rejected = aruco.detectMarkers(
        image=gray,
        dictionary=ARUCO_DICT)
    # If there are markers found, process them
    if ids is not None:
        # Refine detected markers
        corners, ids, rejectedImgPoints, recoveredIds = aruco.refineDetectedMarkers(
            image=gray,
            board=CHARUCO_BOARD,
            detectedCorners=corners,
            detectedIds=ids,
            rejectedCorners=rejected,
            cameraMatrix=None,
            distCoeffs=None)
        
        # Refine detected markers again!
        corners, ids, rejectedImgPoints, recoveredIds = aruco.refineDetectedMarkers(
            image=gray,
            board=CHARUCO_BOARD,
            detectedCorners=corners,
            detectedIds=ids,
            rejectedCorners=rejectedImgPoints,
            cameraMatrix=None,
            distCoeffs=None)
        
        # Interpolate charuco corners
        res, charucoCorners, charucoIds = aruco.interpolateCornersCharuco(
            markerCorners=corners,
            markerIds=ids,
            image=gray,
            board=CHARUCO_BOARD)
        
        # Add interpolated corners and ids to lists
        if charucoCorners is not None and charucoIds is not None:
            allCorners.append(charucoCorners)
            allIds.append(charucoIds)
            
        # Outline all of the markers detected in our image
        img = aruco.drawDetectedMarkers(
            image=img,
            corners=corners)
    # Save image
    cv2.imwrite('./Images/Output/' + fname[9:-4] + '_output.jpg', img)

# Create calibration
imsize = gray.shape
err, cameraMatrix, distCoeffs, rvec, tvec = aruco.calibrateCameraCharuco(
    charucoCorners=allCorners,
    charucoIds=allIds,
    board=CHARUCO_BOARD,
    imageSize=imsize,
    cameraMatrix=None,
    distCoeffs=None)

print(err)
np.savetxt('./cameraMatrix.csv', cameraMatrix, delimiter=',')
np.savetxt('./distCoeffs.csv', distCoeffs, delimiter=',')

mean0 = sum(elt[:][0] for elt in rvec)/len(rvec)
mean1 = sum(elt[:][1] for elt in rvec)/len(rvec)
mean2 = sum(elt[:][2] for elt in rvec)/len(rvec)

means = np.array([mean0, mean1, mean2])

rmat, jac = cv2.Rodrigues(means)

print(rmat)

mean0 = sum(elt[:][0] for elt in tvec)/len(tvec)
mean1 = sum(elt[:][1] for elt in tvec)/len(tvec)
mean2 = sum(elt[:][2] for elt in tvec)/len(tvec)

means = np.array([mean0, mean1, mean2])

print(means)
# Print pixel reprojection error



1.4362803442967436
[[-0.44583135 -0.87827833 -0.17280501]
 [ 0.8793858  -0.39372241 -0.26769999]
 [ 0.1670779  -0.27131133  0.94787929]]
[[ 0.09300829]
 [-0.03127576]
 [ 0.04244235]]
